In [ ]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

open MathNet.Numerics.LinearAlgebra

Installed package MathNet.Numerics.FSharp version 4.15.0

## Варинат 9

In [ ]:
let a = matrix [
    [-401.64; 200.12]
    [1200.72; -601.76]
]

let b = vector [200.; -600.]

1) Найти решение методом Гаусса

In [ ]:
let solveGauss (matrix: Matrix<float>) (freeVector: Vector<float>) = 
    let n = matrix.RowCount
    let extendedMatrix = matrix |> Matrix.appendCol freeVector
    for k = 0 to n - 1 do
        let temp = extendedMatrix.[k, k]
        if abs temp > 1e-16 then
            for j = k + 1 to n do
                extendedMatrix.[k, j] <- extendedMatrix.[k, j] / temp
        for i = k + 1 to n - 1 do
            let temp = extendedMatrix.[i, k]
            for j = k to n do
                extendedMatrix.[i, j] <- extendedMatrix.[i, j] - extendedMatrix.[k, j] * temp 

    let solution = DenseVector.zero<float> n
    for i = n - 1 downto 0 do
        let mutable sum = 0.
        for j = i + 1 to n - 1 do
            sum <- sum + extendedMatrix.[i, j] * solution.[j]
        solution.[i] <- extendedMatrix.[i, n] - sum

    solution  

In [ ]:
let solution = solveGauss a b
printfn "Решение системы методом Гаусса: %A" solution

2) Преобразовать исходную систему к системе вида x = HDx+gD, где HD = E − D−1A,
gD = D−1
b. Здесь D - диагональная матрица, у которой на диагонали находятся
диагональные элементы матрицы A. Вычислить ||H_D||∞.

In [ ]:
let transform (matrix: Matrix<float>) (freeVector: Vector<float>) = 
    let h = DenseMatrix.identity matrix.RowCount - (DenseMatrix.ofDiag <| matrix.Diagonal()).Inverse() * matrix
    let g = (DenseMatrix.ofDiag <| matrix.Diagonal()).Inverse() * freeVector
    h, g

In [ ]:
let (h, g) = transform a b
let norm = h.InfinityNorm()

3) Найти априорную оценку

In [ ]:
let epsilon = 0.001

In [ ]:
let firstEstiomation = DenseVector.zero<float> h.RowCount

In [ ]:
let calculatePrioriEstimation k (h: Matrix<float>) (g: Vector<float>) (firstEstiomation: Vector<float>) =
    h.InfinityNorm() ** k * firstEstiomation.InfinityNorm() +  h.InfinityNorm() ** k / (1. - h.InfinityNorm()) * g.InfinityNorm()

In [ ]:
let getIterationsCount accuracy (h: Matrix<float>) (g: Vector<float>) (firstEstiomation: Vector<float>) = 
    let mutable iterationsCount = 1.
    while calculatePrioriEstimation iterationsCount h g firstEstiomation >= accuracy do 
        iterationsCount <- iterationsCount + 1.

    int iterationsCount

In [ ]:
printfn "Для получения решения с априорной оценкой %f необходимо %i итераций" 
    <| epsilon
    <| getIterationsCount epsilon h g firstEstiomation

Для получения решения с априорной оценкой 

0.001000

 необходимо 

1

 итераций

4) Вычислить решение методом простой итерации с точностью

In [ ]:
let solveIterative step accuracy (h: Matrix<float>) (g: Vector<float>) = 
    let mutable previousEstimation = firstEstiomation
    let mutable currentEstimation: Vector<float> = step h g previousEstimation
    let mutable iterationCount = 1
    while (previousEstimation - currentEstimation).L2Norm() >= accuracy do
        previousEstimation <- currentEstimation
        currentEstimation <- step h g currentEstimation
        iterationCount <- iterationCount + 1
    
    currentEstimation, iterationCount

In [ ]:
let solveSimpleIteration (h: Matrix<float>) (g: Vector<float>) =
    let step (h: Matrix<float>) (g: Vector<float>) (previousX: Vector<float>) = 
        h * previousX + g

    solveIterative step epsilon h g

In [ ]:
let (solution, iterationCount) = solveSimpleIteration h g
printfn "Решение системы методом простой итерации: %A" solution

Решение системы методом простой итерации: 

seq [-0.1994224163; 0.5982672489]

5) Вычислить решение системы методом Зейделя

In [ ]:
let solveSeidel (h: Matrix<float>) (g: Vector<float>) =
    let step (h: Matrix<float>) (g: Vector<float>) (previousX: Vector<float>) = 
        let x = DenseVector.zero<float> previousX.Count
        for i = 0 to x.Count - 1 do
            x.[i] <- g.[i]
            for j = 0 to i - 2 do
                x.[i] <- x.[i] + h.[i, j] * x.[i]
            for j = i - 1 to x.Count - 1 do
                x.[i] <- x.[i] + h.[i, j] * previousX.[i]
        x

    solveIterative step epsilon h g